# Exemple d'analyse d'un dataset : Quartiers a Boston

**Exemple de régressions**

## Importations des librairies courantes

In [ ]:
# Directive pour afficher les graphiques dans Jupyter (inutile si on utilise Spyder)
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score

## Lecture du fichier

On dispose des statistiques de vente des maisons du disctrict de Seattle 

- CRIM : Taux de criminalité
- ZN : Zone résidentielle
- INDUS : Zone industriel
- CHAS : Près de la rivière ou pas
- NOX : Taux de pollution au Nox
- RM : Nombre de pièces
- Age : Age moyen de la maison
- DIS : Distance au centre-ville
- RAD : Nombre de routes principales
- TAX : Taxe habitation
- PT : Nombre d'élèves par prof
- B : Population noire du quartier
- LSTAT : Population pauvre
- MV : Valeur Médiane des prix du quartier en milliers de dollars

In [ ]:
#lecture du dataset
df = pd.read_csv("../input/boston.csv")

In [ ]:
df.head(10).T

In [ ]:
df.info()

In [ ]:
df.info()

In [ ]:
df.count()

## Recherche de corrélations

On a vu assez clairement la corrélation entre le nombre de pièces et la valeur.  
On va utiliser la fonction *corr* pour calculer systématiquement le degré de corrélation entre deux paramètres :

In [ ]:
tabcorr = df.corr()     # on peut utiliser aussi bos.corr(method='pearson') par exemple

Pour visualiser l'ensemble du tableau de corrélations, on utilise une "carte de températures" (*heatmap*) :

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(abs(tabcorr), cmap="coolwarm")

On peut aussi regrouper les paramètres par *clusters* classés par proximité :

In [ ]:
sns.clustermap(abs(tabcorr), cmap="coolwarm")

On peut tracer uniquement le **dendrogramme** des corrélations entre les caractéristiques. On utilise une forme "condensée" de la matrice de corrélation. *linkage* permet de calculer les distances entre caractéristiques à partir des corrélations

In [ ]:
from scipy.cluster import hierarchy as hc

corr = 1 - df.corr()
corr_condensed = hc.distance.squareform(corr)
link = hc.linkage(corr_condensed, method='ward')
plt.figure(figsize=(12,12))
den = hc.dendrogram(link, labels=df.columns, orientation='left', leaf_font_size=10)

On s'intéresse plus précisément à la valeur des maisons :

In [ ]:
correlations = tabcorr.MV
print(correlations)

On élimine la ligne MV elle-même (qui est forcément à 1) :

In [ ]:
correlations = correlations.drop(['MV'],axis=0)

Les corrélations fortement négatives sont aussi significatives que les positives ; on considère donc les valeurs absolues, et on trie par ordre décroissant :

In [ ]:
print(abs(correlations).sort_values(ascending=False))

In [ ]:
sns.distplot(df.MV)

In [ ]:
sns.jointplot(df.LSTAT, df.MV, kind='kde')

On voit qu'il y a des valeurs concentrées sur 50, ces valeurs doivent être supérieures mais calé dessus.
On va les supprimer


# Régression linéaire multiple

On sépare les caractéristiques continues et discrètes :

In [ ]:
df.columns

In [ ]:
continuous_features = ['CRIM', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PT', 'B', 'LSTAT', 'MV']
discrete_features = ['ZN', 'CHAS', 'RAD']

Pour la régression linéaire, on se limite aux quartiers avec une valeur avec un MV < 50, et on élimine les *features* discrètes :

In [ ]:
df1 = df[df["MV"]<50].drop(discrete_features, axis=1)

In [ ]:
X = df1.drop(['MV'], axis=1)
y = df1.MV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

On utilise la fonction de régression linéaire multiple de *sklearn* :

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)            # apprentissage
y_pred = lm.predict(X_test)         # prédiction sur l'ensemble de test

On trace le nuage de points pour comparer la prédiction et les résultats attendus :

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

ou on peut visualiser la distribution de l'erreur avec *seaborn* :

In [ ]:
sns.distplot(y_test-y_pred)

On peut calculer l'erreur sur les moindres carrés :

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_pred)))

Ou le *score R2* (rapport des variances estimée/réelle) :
(https://fr.wikipedia.org/wiki/Coefficient_de_d%C3%A9termination)

In [ ]:
scoreR2 = r2_score(y_test, y_pred)
print(scoreR2)

qu'on peut écrire plus simplement :

In [ ]:
lm.score(X_test,y_test)

## Régression par forêts aléatoires

On va utiliser les forêts aléatoires pour la régression. On conserve l'ensemble des caractéristiques

In [ ]:
X = df1.drop(['MV'], axis=1)
y = df1.MV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestRegressor(random_state=1, n_estimators=100)
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(rf.score(X_test,y_test))

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
sns.distplot(y_test-y_rf)

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_rf)))

**Exercice** : donner l'importance des caractéristiques

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBRegressor()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
print(xgb.score(X_test,y_test))

plt.figure(figsize=(12,12))
plt.scatter(y_test, y_xgb)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")